# Sales Tax Inflation Adjustment

1. Adjusts all the years after 2019 for inflation with 2019 as the baseline
2. Brings data into graphing format
    - If you just want to graph, jump to the graphing section and run.

In [48]:
import pandas as pd
import cpi
from datetime import date

#pd.set_option('display.max_rows', 1000) 
#pd.set_option('display.max_columns', 1000)
#pd.set_option("display.max_colwidth", None) 

In [33]:
cpi.update()

In [61]:
sales_tax = pd.read_csv("sales_tax_2025Q1.csv")
sales_tax.head()

,Source.Name,neighborhood,corridors,cross street,address_start,address_end,2018Q1,2018Q2,2018Q3,2018Q4,...,2022Q4,2023Q1,2023Q2,2023Q3,2023Q4,2024Q1,2024Q2,2024Q3,2024Q4,2025Q1
0,3rd,Bayview,3rd Street,Evans Street to Paul Avenue,3800 3rd St,6190 3rd St,"122,485","97,109","137,167","118,745",...,"108,980","95,319","105,087","99,229","92,757","100,438","92,528","84,797","108,128","95,462"
1,Mission-Excelsior,Excelsior,Mission Street,Silver Avenue to Guttenberg Street,4300 Mission St,5495 Mission St,"186,788","186,454","213,489","234,760",...,"262,496","234,964","226,065","238,863","221,181","222,613","211,213","200,539","210,315","183,578"
2,16th,Mission,16th Street,Dolores Street to Capp Street,2960 16th St,3259 16th St,"91,609","83,718","121,784","107,165",...,"84,375","78,420","93,965","100,005","96,069","95,335","89,863","88,922","88,977","86,831"
3,24th,Mission,24th Street,Valencia Street to Potrero Avenue,2700 24th St,3400 24th St,"74,540","67,691","104,152","90,463",...,"102,150","95,768","105,143","98,168","103,051","106,677","113,078","114,369","107,159","104,417"
4,Mission-Mission,Mission,Mission Street,13th Street and Cesar Chavez Street,1700 Mission St,3400 Cesar Chavez St,"437,171","418,149","527,277","499,922",...,"450,358","433,239","450,711","447,978","454,851","411,061","444,763","418,536","454,984","406,428"


In [62]:
sales_tax["neighborhood"].value_counts()

neighborhood
Sunset        6
Mission       4
Richmond      4
Bayview       1
Excelsior     1
Tenderloin    1
Name: count, dtype: int64

In [63]:
sales_tax.columns = sales_tax.columns.str.strip()

In [64]:
sales_tax = sales_tax.drop(columns = ["2018Q1", "2018Q2","2018Q3","2018Q4"])

In [65]:
quarter_columns = [column for column in mission.columns if column.startswith(('2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025'))]

# Convert all quarter columns to numeric
for column in quarter_columns:
    sales_tax[column] = sales_tax[column].str.replace(",","")
    sales_tax[column] = pd.to_numeric(sales_tax[column])

In [66]:
sales_tax.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Source.Name    17 non-null     object
 1   neighborhood   17 non-null     object
 2   corridors      17 non-null     object
 3   cross street   17 non-null     object
 4   address_start  16 non-null     object
 5   address_end    16 non-null     object
 6   2019Q1         17 non-null     int64 
 7   2019Q2         17 non-null     int64 
 8   2019Q3         17 non-null     int64 
 9   2019Q4         17 non-null     int64 
 10  2020Q1         17 non-null     int64 
 11  2020Q2         17 non-null     int64 
 12  2020Q3         17 non-null     int64 
 13  2020Q4         17 non-null     int64 
 14  2021Q1         17 non-null     int64 
 15  2021Q2         17 non-null     int64 
 16  2021Q3         17 non-null     int64 
 17  2021Q4         17 non-null     int64 
 18  2022Q1         17 non-null     i

In [67]:
quarter_to_month = {1: 3, 2: 6, 3: 9, 4: 12}

for column in quarter_columns:
    year = int(column[:4]) # output will be like "2018"
    quarter = int(column[5]) # output will be like "2"
    month = quarter_to_month[quarter] 
    
    # Create new column with 'adj_' prefix
    # Adjust to the SAME quarter but in 2019
    sales_tax[f'adj_{column}'] = sales_tax[column].apply(lambda x: cpi.inflate(x, date(year, month, 28), to=date(2019, month, 28)))

In [68]:
sales_tax_adj = sales_tax

,Source.Name,neighborhood,corridors,cross street,address_start,address_end,2019Q1,2019Q2,2019Q3,2019Q4,...,adj_2022Q4,adj_2023Q1,adj_2023Q2,adj_2023Q3,adj_2023Q4,adj_2024Q1,adj_2024Q2,adj_2024Q3,adj_2024Q4,adj_2025Q1
0,3rd,Bayview,3rd Street,Evans Street to Paul Avenue,3800 3rd St,6190 3rd St,106927,103254,106775,104182,...,94357.512104,80276.310440,88221.912303,82777.288373,77706.432416,81744.875568,75436.936434,69052.723978,88040.698569,75880.885569
1,Mission-Excelsior,Excelsior,Mission Street,Silver Avenue to Guttenberg Street,4300 Mission St,5495 Mission St,185652,233670,275524,252287,...,227275.367015,197883.349660,189784.527153,199260.613657,185292.607871,181181.146428,172199.352141,163304.883591,171244.076646,145922.578732
2,16th,Mission,16th Street,Dolores Street to Capp Street,2960 16th St,3259 16th St,101435,100916,105628,109507,...,73053.909743,66044.212221,78884.847694,83424.631143,80481.033839,77591.625802,73264.194825,72411.834400,72447.444109,69020.271677
3,24th,Mission,24th Street,Valencia Street to Potrero Avenue,2700 24th St,3400 24th St,84873,101604,92705,90531,...,88443.933396,80654.451875,88268.925037,81892.197291,86330.148312,86822.697495,92191.097809,93134.084798,87251.712951,82999.040754
4,Mission-Mission,Mission,Mission Street,13th Street and Cesar Chavez Street,1700 Mission St,3400 Cesar Chavez St,455245,476194,473735,461221,...,389930.816996,364867.743669,378377.785228,373705.308838,381047.775273,334555.947908,362609.784703,340826.336815,370460.095423,323061.705809


In [78]:
sales_tax_adj = sales_tax[["Source.Name", "neighborhood", "corridors", "cross street", "address_start",
                           "address_end"] +
                           [column for column in sales_tax.columns if column.startswith('adj')]]

In [80]:
sales_tax_adj.head()

,Source.Name,neighborhood,corridors,cross street,address_start,address_end,adj_2019Q1,adj_2019Q2,adj_2019Q3,adj_2019Q4,...,adj_2022Q4,adj_2023Q1,adj_2023Q2,adj_2023Q3,adj_2023Q4,adj_2024Q1,adj_2024Q2,adj_2024Q3,adj_2024Q4,adj_2025Q1
0,3rd,Bayview,3rd Street,Evans Street to Paul Avenue,3800 3rd St,6190 3rd St,106927,103254,106775,104182,...,94357.512104,80276.310440,8.822191e+04,82777.288373,77706.432416,81744.875568,7.543694e+04,69052.723978,88040.698569,7.588089e+04
1,Mission-Excelsior,Excelsior,Mission Street,Silver Avenue to Guttenberg Street,4300 Mission St,5495 Mission St,185652,233670,275524,252287,...,227275.367015,197883.349660,1.897845e+05,199260.613657,185292.607871,181181.146428,1.721994e+05,163304.883591,171244.076646,1.459226e+05
2,16th,Mission,16th Street,Dolores Street to Capp Street,2960 16th St,3259 16th St,101435,100916,105628,109507,...,73053.909743,66044.212221,7.888485e+04,83424.631143,80481.033839,77591.625802,7.326419e+04,72411.834400,72447.444109,6.902027e+04
3,24th,Mission,24th Street,Valencia Street to Potrero Avenue,2700 24th St,3400 24th St,84873,101604,92705,90531,...,88443.933396,80654.451875,8.826893e+04,81892.197291,86330.148312,86822.697495,9.219110e+04,93134.084798,87251.712951,8.299904e+04
4,Mission-Mission,Mission,Mission Street,13th Street and Cesar Chavez Street,1700 Mission St,3400 Cesar Chavez St,455245,476194,473735,461221,...,389930.816996,364867.743669,3.783778e+05,373705.308838,381047.775273,334555.947908,3.626098e+05,340826.336815,370460.095423,3.230617e+05
5,Valencia,Mission,Valencia Street,Duboce Avenue to Cesar Chavez Street,200 Valencia St,3498 Cesar Chavez St,587585,612327,599164,644025,...,406284.502889,369461.867332,3.924254e+05,401868.056175,379581.726816,358970.024576,3.672700e+05,403681.196691,358531.675088,3.994219e+05
6,Balboa,Richmond,Balboa Street,32nd Avenue to 39 Avenue,3101 Balboa St.,3750 Balboa St.,30682,30214,32579,30061,...,23653.449011,26338.519421,2.869876e+04,26778.801903,30975.509542,22805.841995,3.327760e+04,57572.302470,6464.959554,3.017684e+04
7,Clement-Arguello,Richmond,Clement Street,Arguello Blvd to Funston Avenue,1 Clement St.,1156 Clement St.,203046,199893,194679,199868,...,182441.262668,171380.604003,1.773312e+05,171457.408504,183674.090374,162693.772639,1.607331e+05,163808.953736,174925.195304,1.406501e+05
8,Clement-18th Ave,Richmond,Clement Street,18th Avenue to 27th Avenue,1715 Clement St.,2543 Clement St.,58438,57937,58902,74868,...,56689.833961,54592.169403,5.579068e+04,56959.490170,55383.121997,53109.996766,5.870805e+04,57723.767812,53657.535844,5.447396e+04
9,Geary,Richmond,Geary Blvd,Arguello Blvd to 28th Avenue,3700 Geary Blvd.,6350 Geary Blvd.,582184,631410,662892,640203,...,550220.835494,511499.875084,5.323697e+05,574059.564728,558526.763739,523077.513627,5.121164e+05,481928.518432,573257.082980,5.084565e+05


In [84]:
#sales_tax_adj.to_csv("adjusted_sales_tax_2025Q1.csv", index = False)

## Graph formatting

In [125]:
sales_tax_adj["neighborhood"].value_counts()

neighborhood
Sunset        6
Mission       4
Richmond      4
Bayview       1
Excelsior     1
Tenderloin    1
Name: count, dtype: int64

In [ ]:
sales_tax_adj = pd.read_csv("adjusted_sales_tax_2025Q1.csv")
sales_tax_adj.head()

In [121]:
def graph(neighborhood_name):
    
    """
    Take the name of the neighborhood and return a formatted dataframe
    
    """
    
    viz_df = sales_tax_adj[sales_tax_adj["neighborhood"] == neighborhood_name]
    viz_df = viz_df.drop(columns = ["Source.Name", "neighborhood", "cross street", "address_start", "address_end"])
    viz_df.columns = viz_df.columns.str.replace('adj_', '').str.replace('Q', ' Q')
    
    return viz_df

In [122]:
mission = graph("Mission")

sunset = graph("Sunset")

richmond = graph("Richmond")

bayview = graph("Bayview")

excelsior = graph("Excelsior")

tenderloin = graph("Tenderloin")

In [123]:
mission.to_clipboard(index=False)

In [127]:
sunset.to_clipboard(index=False)

In [128]:
richmond.to_clipboard(index=False)

In [129]:
bayview.to_clipboard(index=False)

In [130]:
excelsior.to_clipboard(index=False)

In [131]:
tenderloin.to_clipboard(index=False)